In [1]:
import pandas as pd
import zstandard as zstd
import io
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [ ]:
CALENDAR_DATA_V2_PATH = "../data/raw/v2/calendar.csv.zst"
LISTINGS_DATA_V2_PATH = '../data/raw/v2/listings.csv.zst'
REVIEWS_DATA_V2_PATH = '../data/raw/v2/reviews.csv.zst'
SESSIONS_DATA_V2_PATH = '../data/raw/v2/sessions.csv.zst'
USERS_DATA_V2_PATH = '../data/raw/v2/users.csv.zst'



In [ ]:
import pandas as pd
import zstandard as zstd
import io
from tqdm import tqdm  # for progress tracking (optional)

def read_large_csv_zst(file_path, chunksize=100000, **kwargs):
    # Initialize decompression objects
    dctx = zstd.ZstdDecompressor()

    # Open the compressed file
    with open(file_path, 'rb') as fh:
        # Create a streaming decompressor
        with dctx.stream_reader(fh) as reader:
            # Wrap with a TextIOWrapper for text processing
            text_stream = io.TextIOWrapper(reader, encoding='utf-8')

            # Read the first chunk to get column names
            first_chunk = pd.read_csv(text_stream, nrows=1, **kwargs)

            # Rewind the stream (by reopening)
            fh.seek(0)
            reader = dctx.stream_reader(fh)
            text_stream = io.TextIOWrapper(reader, encoding='utf-8')

            # Initialize list to hold chunks
            chunks = []

            # Create a chunk iterator
            chunk_iter = pd.read_csv(
                text_stream,
                chunksize=chunksize,
                **kwargs
            )

            # Process chunks with optional progress bar
            for chunk in tqdm(chunk_iter, desc="Processing chunks"):
                chunks.append(chunk)

    # Concatenate all chunks
    return pd.concat(chunks, ignore_index=True)